In [1]:
%autosave 0
%matplotlib inline
%config IPCompleter.greedy=True
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.precision',3)
pd.options.display.float_format = '{:,.2f}'.format

import numpy as np
import xlwings as xw
from pathlib import Path
from tqdm.auto import tqdm
from typing import List,Tuple
from datetime import datetime

import ipywidgets as widgets
import re
from IPython.display import display, HTML,Markdown



def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

Autosave disabled


In [2]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

## Getting Data

In [3]:

def get_giro(path)->pd.DataFrame:
    if type(path) is str:
        path = Path(path)
    files = []
    for f in path.glob('*'):
        files.append(f)
    
    df = pd.DataFrame()
    t =  tqdm(files,desc='Loading Reports')
    for f in t:
        if f.suffix == '.csv':
            t.set_postfix_str(f'Loading: {f.name}')
            data = pd.read_csv(f,
                               delimiter=';',
                               decimal=',',
                               thousands ='.',
                               encoding='utf-8',
                               float_precision=3,
                               parse_dates = ['﻿Buchung','Valuta'],
                               dtype={
                                   'Betrag in EUR':float
                               }
                              ).rename(columns={'﻿Buchung':'Buchung'})
            data['Datei_name'] = f.name
            data['Datei_datum'] = datetime.fromtimestamp(f.stat().st_mtime)
            df = df.append(data,sort=True,ignore_index=True)
    df['BuchungsKey'] = df['Buchung'].astype(str) + '-' + df['IBAN / Konto-Nr.'] + '-' + df['Verwendungszweck']
    return df
        

df_giro = get_giro('./Giro')

In [4]:
def get_newest(df:pd.DataFrame,sortcol:str,splitcol:str,keycol:str)->pd.DataFrame:
    """Will split the DF at unique sort+split. 
    Then sort everything after sort and build a new DF, where each key is only taken from one sort+split combo.
    """
    splitdic = {}
    for split,sort in zip(df[splitcol].unique(),df[sortcol].unique()):#loop every unique combination
        splitdic[sort]=(split,df.loc[df[splitcol]==split])
    
    out_df = pd.DataFrame()  #initialize output
    for time,data in  sorted(splitdic.items(), reverse=True):  #loop dict sorted by key
        filename, df = data #unpack sel and data
        if out_df.empty:
            out_df = df # first iteration fill
        else:
            keys = out_df[keycol].unique() #get keys already present in out_df
            df = df.loc[~df[keycol].isin(keys)] # Filter out all already added keys
            out_df = out_df.append(df,sort=True,ignore_index=True)
            
    return out_df
        
df_unique = get_newest(df_giro,'Datei_datum','Datei_name','BuchungsKey')

In [5]:
df_unique.dtypes

BIC / BLZ             object        
Betrag in EUR         float64       
Buchung               datetime64[ns]
BuchungsKey           object        
Buchungstext          object        
Datei_datum           datetime64[ns]
Datei_name            object        
IBAN / Konto-Nr.      object        
Kategorie             object        
Sender / Empfänger    object        
Stichwörter           object        
Umsatz geteilt        object        
Valuta                datetime64[ns]
Verwendungszweck      object        
dtype: object

In [6]:
main_categorys = {
    'Anwaltskosten': 'Misc',
    'Arzneimittel & Medizinprodukte': 'Living',
    'Berufsunfähigkeitsversicherung': 'Insurance',
    'Bildung (Sonstiges)': 'Education',
    'Bücher, Zeitschriften & Spiele': 'Recreation',
    'Computer- und Videospiele': 'Recreation',
    'Einkäufe & Dienstleistungen (Sonstiges)': 'Misc',
    'Einnahmen (Sonstiges)': 'Income',
    'Elektronik & Computer': 'Misc',
    'Essen & Trinken (Sonstiges)': 'Food',
    'Finanzierung & Tilgung': 'Living',
    'Flüge, Autos & Beförderung': 'Transportaion',
    'Freizeit & Sport (Sonstiges)': 'Recreation',
    'Friseur & Körperpflege': 'Living',
    'Gebühren & Bußgelder': 'Misc',
    'Gebühren & Zinsen': 'Misc',
    'Gehalt & Lohn': 'Income',
    'Geldautomat & Barauszahlung': 'Misc',
    'Geschenke & Spenden': 'Misc',
    'Geschäftsreisen': 'Misc',
    'Gesundheit & Pflege (Sonstiges)': 'Living',
    'Getränke': 'Food',
    'Haftpflichtversicherung': 'Insurance',
    'Haushaltsgeräte & Einrichtung': 'Living',
    'Hobbies': 'Recreation',
    'Hotels & Unterkunft': 'Recreation',
    'Internetdienstleistungen': 'Misc',
    'Kfz: Service & Zubehör': 'Transportaion',
    'Kfz: Stellplatz': 'Transportaion',
    'Kfz: Steuer': 'Transportaion',
    'Kfz: Versicherung': 'Transportaion',
    'Kino, Theater & Events': 'Recreation',
    'Kleidung & Schuhe': 'Living',
    'Kommunikation & Unterhaltung': 'Living',
    'Kultur & Unterhaltung (Sonstiges)': 'Recreation',
    'Lebensmittel': 'Food',
    'Leihgaben': 'Misc',
    'Miete': 'Living',
    'Miete & Pacht': 'Living',
    'Mobilfunk': 'Living',
    'Musik & Instrumente': 'Recreation',
    'Musik, Filme & Apps': 'Living',
    'Outdoor & Camping': 'Recreation',
    'Post & Pakete': 'Misc',
    'Renovierung & Instandhaltung': 'Living',
    'Restaurants & Cafes': 'Food',
    'Schmuck & Accessoires': 'Recreation',
    'Schul- & Büromaterial': 'Education',
    'Software': 'Misc',
    'Sparen': 'Savings',
    'Sparen & Anlegen (Sonstiges)': 'Savings',
    'Sport': 'Recreation',
    'Staatliche Hilfe': 'Income',
    'Steuerberatung': 'Misc',
    'Steuererstattungen': 'Income',
    'Treibstoff': 'Transportaion',
    'Umbuchung zwischen Konten ': 'Misc',
    'Unkategorisierte Ausgaben': 'Misc',
    'Unkategorisierte Einnahmen': 'Income',
    'Unterricht, Studiengebühren & Kurse': 'Education',
    'Urlaub: Essen & Trinken': 'Food',
    'Urlaub: Freizeit & Unterhaltung': 'Recreation',
    'Vereine & Mitgliedschaften': 'Recreation',
    'Wellness & Sauna': 'Recreation',
    'Wohnen: Nebenkosten': 'Living',
    'Wohnen: Reinigung': 'Living',
    'Workout & Fitness': 'Recreation',
    'Zins- & Kapitalerträge': 'Income',
    'Zubuchung': 'Income',
    'konsumgüter': 'Recreation',
    'Öffentliche Verkehrsmittel & Taxi': 'Transportaion'
}



In [7]:
df_maincats = df_unique.copy()
df_maincats = df_maincats.merge(pd.DataFrame( [{'Cat':k,'MainCat':v} for k,v in main_categorys.items()]),left_on='Kategorie',right_on='Cat',how='left')

In [8]:
#Input:
granularity = 'Y'
start_date = '01-01-2019'
end_date = datetime.today() #'06-01-1997'

##Processing
piv_data = df_maincats.copy()
datemask = (piv_data['Buchung'] > start_date) & (piv_data['Buchung'] <= end_date)

piv_data = piv_data[datemask]

piv = pd.pivot_table(piv_data, values='Betrag in EUR', index=['MainCat','Kategorie'], columns=pd.Grouper(freq=granularity, key='Buchung'), aggfunc='sum').dropna(how='all') # aggregate counts


In [9]:
[i for i in set(main_categorys.values())]

['Income',
 'Food',
 'Insurance',
 'Living',
 'Misc',
 'Savings',
 'Recreation',
 'Transportaion',
 'Education']

In [15]:
maincat = widgets.SelectMultiple(
    options=[i for i in set(main_categorys.values())],
    value=[i for i in set(main_categorys.values())],
    rows=20,
    description='MainCategorys',
    disabled=False
)
smallcat = widgets.SelectMultiple(
    options=[i for i in set(main_categorys.keys())],
    value=[i for i in set(main_categorys.keys())],
    rows=20,
    description='SmallCategorys',
    disabled=False
)


def on_change(maincat,smallcat):
    display(piv.loc[(piv.index.isin(maincat,level='MainCat'))&(piv.index.isin(smallcat,level='Kategorie'))])


# Updates the image options based on directory value
def update_smallcats(*args):
    oldopts = smallcat.options
    newopts =  [k for k,v in main_categorys.items() if v in maincat.value]
    oldsel = smallcat.value
    smallcat.options = newopts
    newsel = [i for i in list(oldsel) + [i for i in newopts if i not in oldopts] if i in newopts]
    smallcat.value = newsel
    

# Tie the image options to directory value
maincat.observe(update_smallcats, 'value')

mywid = widgets.interactive(on_change, maincat=maincat, smallcat=smallcat)
controls = widgets.HBox(mywid.children[:-1], layout=widgets.Layout(flex_flow='row wrap'))
output = mywid.children[-1]



display(widgets.VBox([controls, output]))

def on_button_clicked(b):
    


button = widgets.Button(description="Alle")
button.on_click(on_button_clicked)